In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'geum_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)
#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2015.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/장계_2015.xlsx
data/geum/자동/장계_2

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0209 - disc_loss: 0.3718 - rmse: 0.2522 - val_loss: 0.3539
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5997 - disc_loss: 0.3533 - rmse: 0.3345 - val_loss: 0.2859
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9178 - disc_loss: 0.3524 - rmse: 0.2800 - val_loss: 0.5690
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 32.4700 - disc_loss: 0.3512 - rmse: 0.8548 - val_loss: 0.3260
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 15.8171 - disc_loss: 0.3257 - rmse: 0.4590 - val_loss: 0.3739
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4559 - disc_loss: 0.3618 - rmse: 0.2942 - val_loss: 0.4349
Epoch 56/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 8.9609 - disc_loss: 0.3224 - rmse: 0.3205 - val_loss: 0.3983
Epoch 57/2000
1/1 [====================

Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9859 - disc_loss: 0.2769 - rmse: 0.2738 - val_loss: 0.2576
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7174 - disc_loss: 0.2762 - rmse: 0.2799 - val_loss: 0.2778
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7913 - disc_loss: 0.2773 - rmse: 0.2472 - val_loss: 0.5776
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4064 - disc_loss: 0.2732 - rmse: 0.2891 - val_loss: 0.5405
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9956 - disc_loss: 0.2946 - rmse: 0.2802 - val_loss: 0.3019
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0768 - disc_loss: 0.2683 - rmse: 0.2711 - val_loss: 0.2763
Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.1777 - disc_loss: 0.2930 - rmse: 0.5399 - val_loss: 0.3228
Epoch 116/2000
1/1 

1/1 [==============================] - 0s 21ms/step - gen_loss: 4.6137 - disc_loss: 0.2650 - rmse: 0.2530 - val_loss: 0.2431
Epoch 168/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.9806 - disc_loss: 0.2515 - rmse: 0.2592 - val_loss: 0.2472
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8718 - disc_loss: 0.2615 - rmse: 0.2295 - val_loss: 0.2385
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7833 - disc_loss: 0.2551 - rmse: 0.2757 - val_loss: 0.2465
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 19.9502 - disc_loss: 0.2578 - rmse: 0.7218 - val_loss: 0.2507
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0953 - disc_loss: 0.2551 - rmse: 0.2341 - val_loss: 0.2821
Epoch 173/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7042 - disc_loss: 0.2472 - rmse: 0.2793 - val_loss: 0.2303
Epoch 174/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4967 - disc_loss: 0.2398 - rmse: 0.2101 - val_loss: 0.2288
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1255 - disc_loss: 0.2303 - rmse: 0.2916 - val_loss: 0.2617
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6745 - disc_loss: 0.2684 - rmse: 0.4450 - val_loss: 0.2365
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.5288 - disc_loss: 0.2410 - rmse: 0.4683 - val_loss: 0.2883
Epoch 229/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.4480 - disc_loss: 0.2369 - rmse: 0.2336 - val_loss: 0.4134
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6869 - disc_loss: 0.2406 - rmse: 0.2343 - val_loss: 0.2563
Epoch 231/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2895 - disc_loss: 0.2572 - rmse: 0.2072 - val_loss: 0.2243
Epoch 232/2000
1/1 [==============

...

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.3855 - disc_loss: 0.1835 - rmse: 0.1927 - val_loss: 0.2431
Epoch 1038/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.0554 - disc_loss: 0.1664 - rmse: 0.2256 - val_loss: 0.2347
Epoch 1039/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4116 - disc_loss: 0.1736 - rmse: 0.1701 - val_loss: 0.2412
Epoch 1040/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.8542 - disc_loss: 0.1732 - rmse: 0.3696 - val_loss: 0.1879
Epoch 1041/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 8.3490 - disc_loss: 0.1787 - rmse: 0.3642 - val_loss: 0.2436
Epoch 1042/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2718 - disc_loss: 0.1783 - rmse: 0.1775 - val_loss: 0.1876
Epoch 1043/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.0808 - disc_loss: 0.1741 - rmse: 0.1662 - val_loss: 0.2204
Epoch 1044/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8705 - disc_loss: 0.1782 - rmse: 0.1910 - val_loss: 0.1785
Epoch 1096/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5707 - disc_loss: 0.1676 - rmse: 0.3265 - val_loss: 0.4764
Epoch 1097/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0474 - disc_loss: 0.1805 - rmse: 0.2077 - val_loss: 0.1760
Epoch 1098/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8577 - disc_loss: 0.1883 - rmse: 0.1960 - val_loss: 0.1968
Epoch 1099/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6849 - disc_loss: 0.1712 - rmse: 0.2227 - val_loss: 0.2202
Epoch 1100/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.5391 - disc_loss: 0.1795 - rmse: 0.3698 - val_loss: 0.4565
Epoch 1101/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2595 - disc_loss: 0.1767 - rmse: 0.1833 - val_loss: 0.2003
Epoch 1102/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5751 - disc_loss: 0.1765 - rmse: 0.2223 - val_loss: 0.4217
Epoch 1154/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4817 - disc_loss: 0.1866 - rmse: 0.1927 - val_loss: 0.4125
Epoch 1155/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2272 - disc_loss: 0.1624 - rmse: 0.1943 - val_loss: 0.2361
Epoch 1156/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2055 - disc_loss: 0.1742 - rmse: 0.1772 - val_loss: 0.1950
Epoch 1157/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.4863 - disc_loss: 0.1760 - rmse: 0.2188 - val_loss: 0.2022
Epoch 1158/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6357 - disc_loss: 0.1623 - rmse: 0.1986 - val_loss: 0.2053
Epoch 1159/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.6395 - disc_loss: 0.1659 - rmse: 0.3736 - val_loss: 0.2299
Epoch 1160/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6913 - disc_loss: 0.1783 - rmse: 0.1888 - val_loss: 0.2531
Epoch 1212/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9430 - disc_loss: 0.1587 - rmse: 0.1886 - val_loss: 0.1712
Epoch 1213/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0609 - disc_loss: 0.1793 - rmse: 0.2086 - val_loss: 0.2017
Epoch 1214/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5421 - disc_loss: 0.1562 - rmse: 0.1926 - val_loss: 0.2425
Epoch 1215/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3291 - disc_loss: 0.1635 - rmse: 0.2082 - val_loss: 0.2059
Epoch 1216/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6525 - disc_loss: 0.1729 - rmse: 0.2087 - val_loss: 0.2137
Epoch 1217/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7539 - disc_loss: 0.1680 - rmse: 0.1997 - val_loss: 0.2168
Epoch 1218/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6455 - disc_loss: 0.1684 - rmse: 0.2250 - val_loss: 0.2187
Epoch 1270/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 4.7083 - disc_loss: 0.1700 - rmse: 0.2797 - val_loss: 0.2055
Epoch 1271/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8520 - disc_loss: 0.1846 - rmse: 0.6704 - val_loss: 0.3905
Epoch 1272/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9910 - disc_loss: 0.1664 - rmse: 0.1932 - val_loss: 0.1882
Epoch 1273/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1891 - disc_loss: 0.1684 - rmse: 0.2133 - val_loss: 0.2174
Epoch 1274/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7535 - disc_loss: 0.1747 - rmse: 0.4453 - val_loss: 0.1852
Epoch 1275/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0975 - disc_loss: 0.1676 - rmse: 0.2118 - val_loss: 0.2039
Epoch 1276/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [43]:
print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]


(52608, 36)
tn


In [44]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (36825, 36) val_df.shape :  (10522, 36) test_df.shape: (5261, 36)


In [45]:
#fake_df.shape[0]/8760

In [46]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [49]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  5
out_num_features :  1


In [50]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [51]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/nitrogen/


In [52]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/20
10/10 [==============================] - ETA: 0s - loss: 0.3039 - mean_absolute_error: 0.4475 - nse: -0.0267
Epoch 00001: val_loss improved from inf to 0.15464, saving model to save/best_model.h5
10/10 [==============================] - 8s 819ms/step - loss: 0.3039 - mean_absolute_error: 0.4475 - nse: -0.0267 - val_loss: 0.1546 - val_mean_absolute_error: 0.3204 - val_nse: 0.2369
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.1273 - mean_absolute_error: 0.2766 - nse: 0.5722
Epoch 00002: val_loss did not improve from 0.15464
10/10 [==============================] - 8s 792ms/step - loss: 0.1273 - mean_absolute_error: 0.2766 - nse: 0.5722 - val_loss: 0.2625 - val_mean_absolute_error: 0.4102 - val_nse: -0.3000
Epoch 3/20
10/10 [==============================] - ETA: 0s - loss: 0.0976 - mean_absolute_error: 0.2361 - nse: 0.6757
Epoch 00003: val_loss improved from 0.15464 to 0.14454, saving model to save/best_model.h5
10/10 [==============================] - 

## core / window.py / 

In [42]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  4.165672
predict_day :  5
------------------------
1.4512528 3.4342904 1.8531839
nse3 :  [0.42637318]
pbias3 :  [-13.668674]
save/geum/models/chlorophyll-a/
year : 2015 ~ 2020
run :  range(0, 1)
target :  chl-a
length :  52608
train=7, test=1, val=2
---------------
[0.42637318]
[13.668674]


In [28]:
#train_df.shape, val_df.shape, test_df.shape

In [29]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************            
#                   SSSSSSSS               ******            ******              *****    *****                      
#        SSSS         S  SSS               ******            ******              *****    *****                     
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
